In [44]:
cd /home/chenxupeng/projects/deepshape/

/home/chenxupeng/projects/deepshape


In [50]:
%pylab inline
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import argparse, sys, os, errno
from glob import glob
import pandas as pd
import matplotlib.pyplot as plt
from keras.layers import Conv2D
from keras.models import Model
from keras.layers import Input, Activation
from keras.layers import add, concatenate
import seaborn as sns
import h5py
import keras as K
from keras.callbacks import TensorBoard
from keras.callbacks import EarlyStopping
from keras.initializers import RandomNormal
from keras.callbacks import ModelCheckpoint
from keras.utils.vis_utils import model_to_dot 
from IPython.display import SVG 
from keras.utils import plot_model
from keras.callbacks import EarlyStopping
from msdcnn import *
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
import tensorflow as tf
import matplotlib.pyplot as plt
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" # so the IDs match nvidia-smi
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.99
set_session(tf.Session(config=config))

Populating the interactive namespace from numpy and matplotlib


In [54]:
model = msdcnnmet(num_layers=16, nc_in=3, nc_out=1)
optim = Adam()
model.compile(optimizer=optim, loss=CrossEntropyLoss(model,10), metrics=[binary_accuracy_with_nan,binary_crossentropy_with_nan,MSE(model)])
#model.summary()

In [ ]:
#model.layers[-8].output
#model.get_layer(index = -8).output
#plot_model(model,to_file='models/modelstructure/msdcnn.png',show_shapes=True,show_layer_names=True)

In [56]:
number = 100
with h5py.File('/home/chenxupeng/projects/deepshape/data/new/train_0') as f:
    images_train = f['train_images'][:number]
#with h5py.File('/home/chenxupeng/projects/deepshape/data/new/test_0') as f:
    #images_test= f['test_images'][:number]
with h5py.File('/home/chenxupeng/projects/deepshape/data/new/Spitale_2015_invivo_CDS_0.1') as f:
    y_train = f['y_train'][:number]
    #y_test = f['y_test'][:number]
y_train = np.concatenate([y_train,y_train],axis = 1)
#y_test= np.concatenate([y_test,y_test],axis = 1)
#y size 256
y_train.shape

(100, 256)

In [63]:
#callbacks = [model_checkpoint]
model_checkpoint = ModelCheckpoint('output/msdcnn_3.12.hdf5', monitor='binary_accuracy_with_nan', save_best_only=True)
#EarlyStopping(monitor=binary_accuracy_with_nan, patience=10, verbose=0)
def Model(images_train,y_train,cv,seq_counts):
    model.fit(images_train, y_train, batch_size=32, nb_epoch=5,
              verbose=1, shuffle=True,validation_split=0.2,
              callbacks=[model_checkpoint,EarlyStopping(monitor='CrossEntropyLoss', patience=10, verbose=0),TensorBoard(log_dir='/home/chenxupeng/projects/deepshape/output/tensorboard/unet/log_dir')])
    model.save('output/msdcnn_3.12.hdf5')

In [64]:
for i in range(1):
    Model(images_train[:number,:,:,:],y_train[:number,:],i,number)

Train on 80 samples, validate on 20 samples
Epoch 1/5
80/80 [==============================] - 5s 60ms/step - loss: 7.2670 - binary_accuracy_with_nan: 0.5044 - binary_crossentropy_with_nan: 7.2033 - MSE: 0.0064 - val_loss: 5.9683 - val_binary_accuracy_with_nan: 0.6045 - val_binary_crossentropy_with_nan: 5.8832 - val_MSE: 0.0085
Epoch 2/5
80/80 [==============================] - 5s 59ms/step - loss: 7.0765 - binary_accuracy_with_nan: 0.5051 - binary_crossentropy_with_nan: 6.9747 - MSE: 0.0102 - val_loss: 5.8819 - val_binary_accuracy_with_nan: 0.6045 - val_binary_crossentropy_with_nan: 5.7473 - val_MSE: 0.0135
Epoch 3/5
80/80 [==============================] - 5s 59ms/step - loss: 6.8555 - binary_accuracy_with_nan: 0.5054 - binary_crossentropy_with_nan: 6.6938 - MSE: 0.0162 - val_loss: 5.8132 - val_binary_accuracy_with_nan: 0.6045 - val_binary_crossentropy_with_nan: 5.6021 - val_MSE: 0.0211
Epoch 4/5
80/80 [==============================] - 5s 58ms/step - loss: 6.5995 - binary_accuracy_w

def draw_structure(num_layers):
    input_layer = Input(shape=(128, 128, 3))
    output_layer, msd_layers = mixed_scaled_dense_network(input_layer, num_layers)
    model = Model(inputs=[input_layer], outputs=[output_layer])
    dot = model_to_dot(model, show_shapes=True, show_layer_names=True)
    dot.set_rankdir('same')
    with open('models/model_%s.dot'%(str(num_layers)), 'w') as f:
        f.write(dot.to_string())
    os.system('dot -Tpdf -o models/model.pdf models/model.dot')
def draw_structure(num_layers):
    input_layer = Input(shape=(128, 128, 3))
    output_layer, msd_layers = mixed_scaled_dense_network(input_layer,num_layers)
    model = Model(inputs=[input_layer], outputs=[output_layer])
                           plot_model(model,to_file='models/modelstructure/model_%s.png'(str(num_layers)),show_shapes=True,show_layer_names=True)
for i in range(32):
    draw_structure(i)